In [1]:
# # relative import ../build/fast_kinematics
# import sys
# sys.path.append('../build')
import numpy as np
import time
import torch
import fast_kinematics
import pytorch_kinematics as pk

In [2]:
N = 4096

In [3]:
# fask kinematics model
model = fast_kinematics.FastKinematics("../kuka_iiwa.urdf", N, "lbr_iiwa_link_7")

In [4]:
# pk chain
chain = pk.build_serial_chain_from_urdf(open("../kuka_iiwa.urdf").read(), "lbr_iiwa_link_7")
d = "cuda"
dtype = torch.float32

chain = chain.to(dtype=dtype, device=d)

In [5]:
th = torch.rand(N, 7, dtype=dtype, device=d, requires_grad=True)
all_times = []
J = None
for _ in range(100):
  start_time = time.time()
  J = chain.jacobian(th)
  end_time = time.time()
  all_times.append(end_time - start_time)
print(f"Time taken for jacobian_mixed_frame: {np.mean(all_times)} with std: {np.std(all_times)}")

Time taken for jacobian_mixed_frame: 0.0229028058052063 with std: 0.03874634669933667


In [6]:
# the output of fast_kinematics is column major 1d array, so we need to reshape it
# experimenting to make sure we got it right
flat_array = torch.arange(1,43)
print(flat_array.view(-1,7,6).permute(0,2,1))

tensor([[[ 1,  7, 13, 19, 25, 31, 37],
         [ 2,  8, 14, 20, 26, 32, 38],
         [ 3,  9, 15, 21, 27, 33, 39],
         [ 4, 10, 16, 22, 28, 34, 40],
         [ 5, 11, 17, 23, 29, 35, 41],
         [ 6, 12, 18, 24, 30, 36, 42]]])


In [11]:
# flatten th
th = th.view(N*7)
all_times = []
J2 = None
for _ in range(1000):
  start_time = time.time()
  J2 = model.jacobian_mixed_frame_pytorch(th).view(-1,7,6).permute(0,2,1)
  end_time = time.time()
  all_times.append(end_time - start_time)
print(f"Time taken for fk: {np.mean(all_times)} with std: {np.std(all_times)}")

Time taken for fk: 1.8442869186401368e-05 with std: 2.9497979648121494e-05


In [10]:
# checking to make sure the results are the same
print(J.shape)
print(J2.shape)
assert torch.allclose(J, J2, atol=1e-5)

torch.Size([4096, 6, 7])
torch.Size([4096, 6, 7])


In [52]:
# just for fun numpy version still faster
qpos = np.random.rand(7*N)
qpos = qpos.astype(np.float32)

all_times = []
for _ in range(1000):
  start_time = time.time()
  poses = model.jacobian_mixed_frame(qpos)
  poses = poses.reshape(N,-1,6).transpose(0,2,1)
  end_time = time.time()
  all_times.append(end_time - start_time)
print(f"Time taken for jacobian_mixed_frame: {np.mean(all_times)} with std: {np.std(all_times)}")

Time taken for jacobian_mixed_frame: 0.00036020946502685546 with std: 0.00024479653917321246
